## How to scrape any website with ScraperAI

### Before we start, install the package

In [23]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[autoreload of scraperai.parsers.webpage_classifier failed: Traceback (most recent call last):
  File "/Users/iakov/Documents/PycharmProjects/scraperai/venv/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "/Users/iakov/Documents/PycharmProjects/scraperai/venv/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 500, in superreload
    update_generic(old_obj, new_obj)
  File "/Users/iakov/Documents/PycharmProjects/scraperai/venv/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 397, in update_generic
    update(a, b)
  File "/Users/iakov/Documents/PycharmProjects/scraperai/venv/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 349, in update_class
    if update_generic(old_obj, new_obj):
  File "/Users/iakov/Documents/PycharmProjects/scraperai/venv/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 397, in update_generic
    updat

In [24]:
# ! pip install scraperai

In [25]:
import os
import json

import pandas as pd
from dotenv import find_dotenv, load_dotenv
from tqdm import tqdm

from scraperai.models import WebpageFields, Pagination, WebpageType, ScraperConfig
from scraperai import ParserAI, Scraper
from scraperai.crawlers import SeleniumCrawler

### Step 1. Init crawler

First, we need to initialize a web-crawler that will help us to fetch data from the web.

In this tutorial we use `SeleniumCrawler` that uses Selenium webdriver. By default it creates a new Chrome session.

To use other browsers you can pass your own webdriver (both local and remote) to the `SeleniumCrawler`:
```
crawler = SeleniumCrawler(driver=your_own_webdriver)
```

If you want to use playwright or other services, you can create your own crawler implementation based on `BaseCrawler`.

In [26]:
crawler = SeleniumCrawler()

### Step 2. Init ParserAI

By default, we use the latest OpenAI GPT-4 model. You can place your API key in the `.env` file. If you don't have a key, you can get it [here](https://platform.openai.com/api-keys).
Also, you can use another AI model. To do this, you need to create another implementations of the `BaseJsonLM` and `BaseVision` classes.

In [27]:
env_file = find_dotenv()
if env_file:
    load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key is None:
    openai_api_key = input('Please, enter your OpenAI API key: ')

parser = ParserAI(openai_api_key=openai_api_key)

There are 2 experiments in this doc:
1. [List of YCombinator companies](https://www.ycombinator.com/companies/)
2. [List of commits in the repository](https://github.com/scraperai/scraperai/commits/main/)

#### Experiment 1. List of YCombinator companies

### Step 3. Open the website page
Later, in case of multiple similar sites you will be able to run batch scraping. The main target is to semi-automatically detect all xpaths

In [28]:
url = 'https://www.ycombinator.com/companies' # Enter the URL of the website

In [29]:
# Open page in the browser
crawler.get(url)

### Step 3.1. Detect page type
We divide webpages into 4 categories:
- **Catalog**: consists of similar-looking repeating elements. It can be a list of products, articles, companies, table rows, etc;
- **Details**: contains main information about one product;
- **Captcha**: in case we meet anti-scraping CAPTCHA;
- **Other**: everything else; we don't support this webpage type yet.

By default, we use screenshot of the page and GPT4 Vision model to determine a type. We also have a fallback algorithm if you cannot take a screenshot of the page or do not have access to Vision models.

If you know the type of the page, you can set it manually.

In [30]:
page_type = parser.detect_page_type(
    page_source=crawler.page_source,
    screenshot=crawler.get_screenshot_as_base64()
)
# You can set type manually
# page_type = WebpageType.CATALOG
page_type

<WebpageType.CATALOG: 'catalog'>

OpenAI tokens are spent on each action. You can find total money spent using:

In [10]:
parser.total_cost  # in USD

0.00847

### Step 3.2. Detect pagination
**It is used only for type `catalog`.**

We need to pass a whole page to detect the pagination.
There are 3 types of pagination: `xpath`, `scroll`, and `urls`.

In [31]:
pagination = parser.detect_pagination(crawler.page_source)
pagination

Pagination(type='xpath', xpath="//div[text()='Loading more...']", url_param=None, url_param_first_value=1)

In case of error, you can set it manually.

In [38]:
# Scroll type
pagination = Pagination(type='scroll')
# XPATH
p2 = Pagination(type='xpath', xpath='//some-xpath')
# URLs
p3 = Pagination(type='urls', urls=['https://www.ycombinator.com/companies/airbnb', ])

### Step 3.3. Detect catalog items
**It is used only for type `catalog`.**
You should correctly choose item block, url and ... 
AI isn't perfect, so you can manually add extra prompt to help AI to understand what you want or set xpath manually.

In [32]:
catalog_item = parser.detect_catalog_item(page_source=crawler.page_source, website_url=url, extra_prompt=None)
catalog_item

<card_xpath=//a[contains(@class, '_company_99gj3_339')]
 url_xpath=//a[contains(@class, '_company_99gj3_339')]/@href
 urls_on_page=['https://www.ycombinator.com/companies/airbnb', 'https://www.ycombinator.com/companies/amplitude', 'https://www.ycombinator.com/companies/coinbase', 'https://www.ycombinator.com/companies/doordash', 'https://www.ycombinator.com/companies/dropbox', 'https://www.ycombinator.com/companies/ginkgo-bioworks', 'https://www.ycombinator.com/companies/gitlab', 'https://www.ycombinator.com/companies/instacart', 'https://www.ycombinator.com/companies/matterport', 'https://www.ycombinator.com/companies/pagerduty', 'https://www.ycombinator.com/companies/weave', 'https://www.ycombinator.com/companies/oklo', 'https://www.ycombinator.com/companies/embark-trucks', 'https://www.ycombinator.com/companies/lucira-health', 'https://www.ycombinator.com/companies/notable-labs', 'https://www.ycombinator.com/companies/rigetti-computing', 'https://www.ycombinator.com/companies/pardes

You can highlight fields using selenium:

In [33]:
if catalog_item is not None:
    crawler.highlight_by_xpath(catalog_item.card_xpath, '#8981D7', 5)
    crawler.highlight_by_xpath(catalog_item.url_xpath, '#5499D1', 3)

### Step 3.4. Detect data fields in a catalog item

We define two types of data fields in a HTML page.

First type is static field that do not contain a field name. It can be both a single value or an array. Example: product name or price.

Second type is dynamic fields where there are both field names and values mentioned. Usually these fields look like tables:
param1: value1
param2: value2
etc.

In [34]:
# Aux method to print detected fields
def _print_fields(fields: WebpageFields):
    print(f'Static fields ({len(fields.static_fields)}):')

    data = [{'name': f.field_name, 'xpath': f.field_xpath, 'value': f.first_value} for f in fields.static_fields]
    df = pd.DataFrame(data)
    print(df.to_markdown(tablefmt='plain', index=True))

    print()
    print(f'Dynamic fields ({len(fields.dynamic_fields)}):')
    if len(fields.dynamic_fields) == 0:
        print('Not found')
        return
    index = len(fields.static_fields)
    for field in fields.dynamic_fields:
        print(f' {index}  Section {field.section_name}\n'
                   f'    Labels xpath: {field.name_xpath}\n'
                   f'    Values xpath: {field.value_xpath}\n'
                   f'    Value: {field.first_values}')
        index += 1

In [35]:
fields = parser.extract_fields(html_snippet=catalog_item.html_snippet)
_print_fields(fields)

Static fields (6):
    name          xpath                                                                    value
 0  Company Name  //span[@class='_coName_99gj3_454']                                       Airbnb
 1  Location      //span[@class='_coLocation_99gj3_470']                                   San Francisco, CA, USA
 2  Description   //span[@class='_coDescription_99gj3_479']                                Book accommodations around the world.
 3  Batch         //a[@class='_tagLink_99gj3_1024'][1]/span[@class='pill _pill_99gj3_33']  W09
 4  Industry 1    //a[@class='_tagLink_99gj3_1024'][2]/span[@class='pill _pill_99gj3_33']  Consumer
 5  Industry 2    //a[@class='_tagLink_99gj3_1024'][3]/span[@class='pill _pill_99gj3_33']  Travel, Leisure and Tourism

Dynamic fields (0):
Not found


You can highlight detected fields:

In [36]:
# Method to highlight fields
def highlight_fields(fields: WebpageFields):
    colors = ['#539878', '#5499D1', '#549B9A', '#5982A3', '#5A5499', '#68D5A2', '#75DDDC', '#8981D7', '#98D1FF',
              '#98FFCF', '#9D5A5A', '#A05789', '#AAFFFE', '#C6C1FF', '#CD7CB3', '#D17A79', '#FAB4E4', '#FFB1B0']
    for index, field in enumerate(fields.static_fields):
        crawler.highlight_by_xpath(field.field_xpath, colors[index % len(colors)], border=4)
    for index, field in enumerate(fields.dynamic_fields):
        color = colors[index % len(colors)]
        crawler.highlight_by_xpath(field.value_xpath, color, border=3)
        crawler.highlight_by_xpath(field.name_xpath, color, border=3)

In [37]:
highlight_fields(fields)

### Step 3.4 Scrape data

We are almost there!

First of all, let's set some limits for simplicity:

In [39]:
max_pages = 5  # How many catalog pages we should iterate over
max_rows = 200  # How many rows to scrape before stop

Let's init scraper config that you can reuse later:

In [40]:
config = ScraperConfig(
    start_url=url,
    page_type=page_type,
    pagination=pagination,
    catalog_item=catalog_item,
    open_nested_pages=False,
    fields=fields,
    max_pages=max_pages,
    max_rows=max_rows
)

Now we need to create a Scraper instance using Crawler and ScraperConfig. It will iterate over catalog cards, automatically handle pagination and data-extracting.

In [41]:
scraper = Scraper(config, crawler)

rows = []
for item in tqdm(scraper.scrape(), total=max_rows):
    rows.append(item)

320it [00:22, 14.18it/s]                        


#### Congratulations! We got the final data!

In [42]:
rows

[{'Company Name': 'Airbnb',
  'Location': 'San Francisco, CA, USA',
  'Description': 'Book accommodations around the world.',
  'Batch': None,
  'Industry 1': None,
  'Industry 2': None},
 {'Company Name': 'Amplitude',
  'Location': 'San Francisco, CA, USA',
  'Description': 'Digital Analytics Platform',
  'Batch': None,
  'Industry 1': None,
  'Industry 2': None},
 {'Company Name': 'Coinbase',
  'Location': 'San Francisco, CA, USA',
  'Description': 'Buy, sell, and manage cryptocurrencies.',
  'Batch': None,
  'Industry 1': None,
  'Industry 2': None},
 {'Company Name': 'DoorDash',
  'Location': 'San Francisco, CA, USA',
  'Description': 'Restaurant delivery.',
  'Batch': None,
  'Industry 1': None,
  'Industry 2': None},
 {'Company Name': 'Dropbox',
  'Location': 'San Francisco, CA, USA',
  'Description': 'Backup and share files in the cloud.',
  'Batch': None,
  'Industry 1': None,
  'Industry 2': None},
 {'Company Name': 'Ginkgo Bioworks',
  'Location': 'Boston, MA, USA',
  'Descri

In [43]:
import pandas as pd
pd.DataFrame(rows)

,Company Name,Location,Description,Batch,Industry 1,Industry 2
0,Airbnb,"San Francisco, CA, USA",Book accommodations around the world.,None,None,None
1,Amplitude,"San Francisco, CA, USA",Digital Analytics Platform,None,None,None
2,Coinbase,"San Francisco, CA, USA","Buy, sell, and manage cryptocurrencies.",None,None,None
3,DoorDash,"San Francisco, CA, USA",Restaurant delivery.,None,None,None
4,Dropbox,"San Francisco, CA, USA",Backup and share files in the cloud.,None,None,None
...,...,...,...,...,...,...
315,Asseta,"Newport Beach, CA, USA",Marketplace for industrial spare parts and cap...,None,None,None
316,Per Vices,"Toronto, ON, Canada",Building Mission Critical Software Defined Rad...,None,None,None
317,7cups,"Virginia Beach, VA, USA",7 Cups is an emotional support service that ma...,None,None,None
318,Watsi,"San Francisco, CA, USA",Watsi is the first global crowdfunding platfor...,None,None,None


You can export data in any format:

In [ ]:
# Export as json
with open('yc.json', 'w+') as f:
    json.dump(rows, f, indent=4)

# Export to Pandas DataFrame
df = pd.DataFrame(rows)
df.to_csv('yc.csv')
df

### Step 4. Parse nested detail page

You can extract data from nested pages using ScraperAI

In [ ]:
# Open first nested page
crawler.get(catalog_item.urls_on_page[0])

### Step 4.1. Extract fields

First, we use `summarize_details_page_as_valid_html` method to find relevant parts on the initial webpage.
For example, a list of similar products is not a relevant part of a details page.

Then we use `parser.extract_fields` as before to get the fields from html snippet.

In [ ]:
html_snippet = parser.summarize_details_page_as_valid_html(
    page_source=crawler.page_source,
    screenshot=crawler.get_screenshot_as_base64()
)
fields = parser.extract_fields(html_snippet)
_print_fields(fields)

In [ ]:
# Let's highlight the fields
highlight_fields(fields)

### Step 4.2. Scrape data

First, let's set some limits for simplicity:

In [ ]:
max_pages = 5  # How many catalog pages we should iterate over
max_rows = 20  # How many rows to scrape before stop

We are ready to collect the data:

In [ ]:
config = ScraperConfig(
    start_url=url,
    page_type=page_type,
    pagination=pagination,
    catalog_item=catalog_item,
    open_nested_pages=True,
    fields=fields,
    max_pages=max_pages,
    max_rows=max_rows
)
scraper = Scraper(config, crawler)

rows = []
for item in tqdm(scraper.scrape(), total=max_rows):
    rows.append(item)

In [ ]:
pd.DataFrame(rows)

#### Experiment 2. [List of commits in the repository](https://github.com/scraperai/scraperai/commits/main/)

In [ ]:
# Define url
url = 'https://github.com/scraperai/scraperai/commits/main/'

In [ ]:
# Open url
crawler.get(url)

In [ ]:
# Detect page_type
page_type = WebpageType.CATALOG
page_type

In [ ]:
# Detect pagination
pagination = parser.detect_pagination(crawler.page_source)
pagination

In [ ]:
# Detect catalog item
catalog_item = parser.detect_catalog_item(
    page_source=crawler.page_source,
    website_url=url,
    extra_prompt='This page contains a list of commits. Each commit row is a catalog item')
catalog_item

In [ ]:
crawler.highlight_by_xpath(catalog_item.card_xpath, '#8981D7', 5)
crawler.highlight_by_xpath(catalog_item.url_xpath, '#5499D1', 3)

In [ ]:
fields = parser.extract_fields(html_snippet=catalog_item.html_snippet)
_print_fields(fields)

In [ ]:
highlight_fields(fields)

In [ ]:
config = ScraperConfig(
    start_url=url,
    page_type=page_type,
    pagination=pagination,
    catalog_item=catalog_item,
    open_nested_pages=True,
    fields=fields,
    max_pages=2,
    max_rows=100
)
scraper = Scraper(config, crawler)

rows = []
for item in tqdm(scraper.scrape(), total=max_rows):
    rows.append(item)

In [ ]:
pd.DataFrame(rows)

### That's it!
#### Thank you for using ScraperAI